<img src="../imgs/logo.png" width="20%" align="right" style="margin:0px 20px">


# Evolutionary Computation

## 5.3 Deep Neuroevolution

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-sa/4.0/80x15.png" /></a>&nbsp;| Dennis G. Wilson | <a href="https://d9w.github.io/evolution/">https://d9w.github.io/evolution/</a>

In [6]:
using PyCall
using Conda

In [7]:
Conda.add_channel("conda-forge")

┌ Info: Running `conda config --add channels conda-forge --file /Users/boutryoscar/.julia/conda/3/condarc-julia.yml --force` in root environment
└ @ Conda /Users/boutryoscar/.julia/packages/Conda/3rPhK/src/Conda.jl:113


In [8]:
Conda.add("gym")

┌ Info: Running `conda install -y gym` in root environment
└ @ Conda /Users/boutryoscar/.julia/packages/Conda/3rPhK/src/Conda.jl:113


Solving environment: ...working... done

# All requested packages already installed.



In [9]:
import Random
Random.seed!(1234);

In [10]:
include("cmaes.jl");

In [11]:
struct FCLayer
    w::Array{Float64}
    b::Array{Float64}
end

struct SimpleANN
    l1::FCLayer
    l2::FCLayer
    out::FCLayer
end

In [12]:
function SimpleANN(input::Int, N1::Int, N2::Int, output::Int)
    l1 = FCLayer(zeros(N1, input), zeros(N1))
    l2 = FCLayer(zeros(N2, N1), zeros(N2))
    out = FCLayer(zeros(output, N2), zeros(output))
    SimpleANN(l1, l2, out)
end

SimpleANN

In [13]:
ann = SimpleANN(5, 64, 64, 4);

In [14]:
function compute(inputs::Array{Float64}, ann::SimpleANN)
    x = ann.l1.w * inputs .+ ann.l1.b
    x = ann.l2.w * x .+ ann.l2.b
    x = ann.out.w * x .+ ann.out.b
    x
end

compute (generic function with 1 method)

In [15]:
compute(zeros(5), ann)

4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

In [16]:
gym = pyimport("gym")

PyObject <module 'gym' from '/Users/boutryoscar/.julia/conda/3/lib/python3.7/site-packages/gym/__init__.py'>

In [17]:
env = gym.make("CartPole-v1")
n_in = 4
n_out = 2;

In [18]:
function play_env(ann::SimpleANN; render=false)
    env = gym.make("CartPole-v1")
    env.seed(3)
    obs = env.reset()
    total_reward = 0.0
    done = false
    
    while ~done
        action = argmax(compute(obs, ann))-1
        obs, reward, done, _ = env.step(action)
        if render
            env.render()
        end
        total_reward += reward
    end
    env.close()
    env = nothing
    Base.GC.gc()
    total_reward
end

play_env (generic function with 1 method)

In [14]:
ann = SimpleANN(n_in, 5, 5, n_out)
play_env(ann; render=true)

9.0

In [15]:
play_env(ann)

9.0


signal (15): Terminated: 15
in expression starting at /Users/boutryoscar/.julia/packages/IJulia/DrVMH/src/kernel.jl:52


In [19]:
function genes_to_ann(genes::Array{Float64})
    ann = SimpleANN(n_in, 5, 5, n_out)
    layers = [ann.l1.w, ann.l1.b, ann.l2.w, ann.l2.b, ann.out.w, ann.out.b]
    L = 1
    j = 1
    for i in eachindex(genes)
        if j > length(layers[L])
            L += 1
            j = 1
        end
        layers[L][j] = genes[i]
        j += 1
    end
    ann
end

genes_to_ann (generic function with 1 method)

In [20]:
function objective(genes::Array{Float64})
    ann = genes_to_ann(genes)
    -play_env(ann)
end

objective (generic function with 1 method)

In [21]:
N = n_in*5 + 5 + 5*5 + 5 + 5*n_out + n_out #nb de paramètres à optimiser

67

In [24]:
ann = genes_to_ann(randn(N)) #initialisation aléatoire des poids et biais

SimpleANN(FCLayer([1.2396882940094314 0.05331004567633298 -0.1982166040781264 2.5296080046084493; 1.0615890099967913 1.8600016877104826 1.7177182158527435 -1.5385442124417505; … ; 0.7142514255758221 -0.33240336585565466 0.6415345022224517 -0.34992245953884205; -0.06823765109902417 0.22604552898081612 -1.6715750451349984 0.03248388511060847], [-1.3431639277644547, -1.686258966332342, 0.4285747678322418, -0.6766059259474498, 0.23682745319463824]), FCLayer([1.4478365943622638 -0.5485488997691811 … -0.5962711113219342 -0.3382145297384006; -0.4044408603831723 -0.4447913271327849 … 0.8619387220299867 0.7224798283492364; … ; 0.8844124120235424 1.031116879013816 … 0.14310509125359627 -0.9842166483577759; 0.8803904746548468 -0.9470851049051129 … 0.2952224330631754 1.4564746688140648], [0.08491622170695262, -0.684287467419645, 0.6879327492364012, -0.09386813353618305, 0.24942579193878897]), FCLayer([0.059488150260498335 -1.860087003990038 … -1.1781214641906343 0.5457162608895234; -1.490262786108

In [25]:
play_env(ann)

9.0

In [26]:
play_env(ann)

9.0

In [27]:
c = CMAES(N=N, µ=10, λ=30, τ=sqrt(N), τ_c=N^2, τ_σ=sqrt(N))
for i in 1:5
    step!(c, objective)
    println(i, " ", maximum(.-c.F_λ))
end

1 124.0
2 227.0
3 412.0
4 500.0
5 437.0


In [29]:
best = nothing
best_fit = -Inf
c = CMAES(N=N, µ=10, λ=30, τ=sqrt(N), τ_c=N^2, τ_σ=sqrt(N))
for i in 1:20
    step!(c, objective)
    bestind = argmin(c.F_λ)
    maxfit = -c.F_λ[bestind]
    println(i, " ", maxfit)
    if maxfit > best_fit
        best = copy(c.offspring[bestind])
        best_fit = maxfit
    end
    if best_fit == 500
        break
    end
end

1 34.0
2 46.0
3 68.0
4 194.0
5 74.0
6 500.0


In [30]:
ann = genes_to_ann(best)
play_env(ann; render=true)

500.0